### GrabCut
1️ 초기 사각형 지정
→ 사용자가 대략적으로 전경을 포함하는 사각형을 그림.\
2️ 픽셀 클래스 초기화
→ 사각형 안은 잠재적 전경, 밖은 확실한 배경.\
3️ GMM 학습
→ 전경과 배경 각각의 색상 분포를 GMM으로 추정.\
4️ Graph Cut
→ 현재 GMM 기반으로 전경/배경 분할.\
5️ Iterative Refinement
→ 위 과정 반복해서 GMM을 업데이트하고 경계를 점진적으로 개선.\
6️ 마스크 보정
→ 사용자가 필요하다면 마스크에 선을 추가로 그려서 수정 가능.

In [2]:
import cv2 as cv
import numpy as np

# [1] 이미지 불러오기 & 복사본 만들기
img = cv.imread('soccer.jpg')  # 원본 이미지
img_show = np.copy(img)        # 마우스로 그릴 때 표시용

# [2] GrabCut 마스크 초기화
# 0: 확실한 배경, 1: 확실한 물체, 2: 잠재적 배경, 3: 잠재적 물체
mask = np.zeros((img.shape[0], img.shape[1]), np.uint8)
mask[:, :] = cv.GC_PR_BGD      # 처음엔 전체 잠재적 배경으로 설정

# [3] 붓 크기 & 색상
BrushSize = 9
LColor, RColor = (255, 0, 0), (0, 0, 255)  # 파란색(물체), 빨간색(배경)

# [4] 마우스 이벤트 콜백 함수
def painting(event, x, y, flags, param):
    if event == cv.EVENT_LBUTTONDOWN:
        # 왼쪽 버튼: 물체 그리기
        cv.circle(img_show, (x, y), BrushSize, LColor, -1)  # 표시용
        cv.circle(mask, (x, y), BrushSize, cv.GC_FGD, -1)   # 마스크값은 확실한 물체(1)
    elif event == cv.EVENT_RBUTTONDOWN:
        # 오른쪽 버튼: 배경 그리기
        cv.circle(img_show, (x, y), BrushSize, RColor, -1)
        cv.circle(mask, (x, y), BrushSize, cv.GC_BGD, -1)   # 마스크값은 확실한 배경(0)
    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_LBUTTON:
        # 왼쪽 버튼 누른 채 이동: 물체 계속 그리기
        cv.circle(img_show, (x, y), BrushSize, LColor, -1)
        cv.circle(mask, (x, y), BrushSize, cv.GC_FGD, -1)
    elif event == cv.EVENT_MOUSEMOVE and flags == cv.EVENT_FLAG_RBUTTON:
        # 오른쪽 버튼 누른 채 이동: 배경 계속 그리기
        cv.circle(img_show, (x, y), BrushSize, RColor, -1)
        cv.circle(mask, (x, y), BrushSize, cv.GC_BGD, -1)
        
    cv.imshow('Painting', img_show)  # 그릴 때마다 갱신

# [5] 윈도우 생성 & 마우스 콜백 연결
cv.imshow('Painting', img_show)
cv.setMouseCallback('Painting', painting)

# [6] 'q' 키 누를 때까지 그리기 반복
while(True):
    if cv.waitKey() == ord('q'):
        break

# [7] GrabCut 모델 배열 초기화 (배경/물체 GMM)
background = np.zeros((1, 65), np.float64)
foreground = np.zeros((1, 65), np.float64)

# [8] GrabCut 수행 (마스크 모드)
cv.grabCut(img, mask, None, background, foreground, 5, cv.GC_INIT_WITH_MASK)

# [9] 결과 마스크: 확실한/잠재적 배경은 0, 물체은 1
mask2 = np.where(
    (mask == cv.GC_BGD) | (mask == cv.GC_PR_BGD),
    0, 1
).astype('uint8')

# [10] 원본에 마스크 적용: 물체만 살리기
grab = img * mask2[:, :, np.newaxis]

# [11] 결과 출력
cv.imshow('Grab cut image', grab)

cv.waitKey()
cv.destroyAllWindows()


### 작동흐름
| 단계             | 내용                                                  |
| -------------- | --------------------------------------------------- |
| **이미지 준비**     | 원본 + 마스크 초기화 (`GC_PR_BGD`)                          |
| **마우스 그리기**    | 왼쪽 버튼: 전경(파란색), 오른쪽 버튼: 배경(빨간색)                     |
| **GrabCut 실행** | `GC_INIT_WITH_MASK` 모드: 마스크 기반으로 GMM 학습 & Graph Cut |
| **결과 만들기**     | 전경만 남기고 배경은 제거                                      |
| **출력**         | `cv.imshow`로 최종 결과 확인                               |